In [17]:
import pandas as pd

df = pd.read_csv('jehle_verb_database.csv')

In [18]:
df.head(10)

,infinitive,infinitive_english,mood,mood_english,tense,tense_english,verb_english,form_1s,form_2s,form_3s,form_1p,form_2p,form_3p,gerund,gerund_english,pastparticiple,pastparticiple_english
0,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Presente,Present,"I abandon, am abandoning",abandono,abandonas,abandona,abandonamos,abandonáis,abandonan,abandonando,abandoning,abandonado,abandoned
1,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Futuro,Future,I will abandon,abandonaré,abandonarás,abandonará,abandonaremos,abandonaréis,abandonarán,abandonando,abandoning,abandonado,abandoned
2,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Imperfecto,Imperfect,"I was abandoning, used to abandon, abandoned",abandonaba,abandonabas,abandonaba,abandonábamos,abandonabais,abandonaban,abandonando,abandoning,abandonado,abandoned
3,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Pretérito,Preterite,I abandoned,abandoné,abandonaste,abandonó,abandonamos,abandonasteis,abandonaron,abandonando,abandoning,abandonado,abandoned
4,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Condicional,Conditional,I would abandon,abandonaría,abandonarías,abandonaría,abandonaríamos,abandonaríais,abandonarían,abandonando,abandoning,abandonado,abandoned
5,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Presente perfecto,Present Perfect,I have abandoned,he abandonado,has abandonado,ha abandonado,hemos abandonado,habéis abandonado,han abandonado,abandonando,abandoning,abandonado,abandoned
6,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Futuro perfecto,Future Perfect,I will have abandoned,habré abandonado,habrás abandonado,habrá abandonado,habremos abandonado,habréis abandonado,habrán abandonado,abandonando,abandoning,abandonado,abandoned
7,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Pluscuamperfecto,Past Perfect,I had abandoned,había abandonado,habías abandonado,había abandonado,habíamos abandonado,habíais abandonado,habían abandonado,abandonando,abandoning,abandonado,abandoned
8,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Pretérito anterior,Preterite (Archaic),I had abandoned,hube abandonado,hubiste abandonado,hubo abandonado,hubimos abandonado,hubisteis abandonado,hubieron abandonado,abandonando,abandoning,abandonado,abandoned
9,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Indicative,Condicional perfecto,Conditional Perfect,I would have abandoned,habría abandonado,habrías abandonado,habría abandonado,habríamos abandonado,habríais abandonado,habrían abandonado,abandonando,abandoning,abandonado,abandoned


In [19]:
df.rename(columns={'infinitive': 'spanishName', 'infinitive_english': 'englishName'}, inplace=True)
df.drop(columns=['mood_english', 'tense_english'], inplace=True)
df.rename(columns={'form_1s': 'yo', 'form_2s': 'tu', 'form_3s': 'el', 'form_1p': 'nosotros', 'form_2p': 'vosotros', 'form_3p': 'ellos'}, inplace=True)
df.rename(columns={'verb_english': 'translation'}, inplace=True)
df.drop(columns=['gerund', 'gerund_english', 'pastparticiple', 'pastparticiple_english'], inplace=True)

In [20]:
df.head()

,spanishName,englishName,mood,tense,translation,yo,tu,el,nosotros,vosotros,ellos
0,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Presente,"I abandon, am abandoning",abandono,abandonas,abandona,abandonamos,abandonáis,abandonan
1,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Futuro,I will abandon,abandonaré,abandonarás,abandonará,abandonaremos,abandonaréis,abandonarán
2,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Imperfecto,"I was abandoning, used to abandon, abandoned",abandonaba,abandonabas,abandonaba,abandonábamos,abandonabais,abandonaban
3,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Pretérito,I abandoned,abandoné,abandonaste,abandonó,abandonamos,abandonasteis,abandonaron
4,abandonar,"to abandon, leave behind, desert; to quit, giv...",Indicativo,Condicional,I would abandon,abandonaría,abandonarías,abandonaría,abandonaríamos,abandonaríais,abandonarían


In [ ]:
'''
database schema:
CREATE TABLE IF NOT EXISTS questions (
	id SERIAL PRIMARY KEY,
	englishName TEXT NOT NULL,
	spanishName TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS tenses (
	id SERIAL PRIMARY KEY,
	questionId INTEGER REFERENCES questions(id),
    tense TEXT NOT NULL,
    mood TEXT NOT NULL,
    translation TEXT NOT NULL,
	yo TEXT NOT NULL,
	tu TEXT NOT NULL,
	el TEXT NOT NULL,
	nosotros TEXT NOT NULL,
	vosotros TEXT NOT NULL,
	ellos TEXT NOT NULL
);
'''

In [22]:
import psycopg2 as pg
from dotenv import load_dotenv
import os

load_dotenv()
DB_URL = os.getenv('DB_URL')

conn = pg.connect(DB_URL)

In [23]:
cur = conn.cursor()

# first insert all the questions (note: there are duplicates. make sure the spanishName is unique)
questions = df[['englishName', 'spanishName']].drop_duplicates("spanishName")
questions.to_sql('questions', conn, if_exists='append', index=False)

/var/folders/j3/r33c6vp900n1231zcdrzvk500000gn/T/ipykernel_90683/3751422937.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  questions.to_sql('questions', conn, if_exists='append', index=False)


DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': syntax error at or near ";"
LINE 8:             AND name=?;
                              ^
